In [2]:
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F

In [5]:
class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__(in_features,out_features, bias)    
    def set_mask(self, mask: torch.Tensor):
        self.register_buffer("mask", mask)

    def forward(self, X):
        return F.linear(X, self.weight * self.mask, self.bias)

In [ ]:
# Made w/ mixture of logitsics 

class MADE(nn.Module):
    def __init__(self,in_shape,num_logistics,num_classes, hidden_sizes = [1024, 1024, 1024]):
        super(MADE, self).__init__()
        self.in_shape = self.in_shape
        self.flatten_shape = np.prod(self.in_shape)
        self.hidden_sizes = hidden_sizes

        self.num_logistics = num_logistics
        self.num_classes = num_classes

        self.pi = torch.randn(self.num_heads)
        self.means = torch.arange(self.num_heads)
        self.scales = torch.zeros(self.num_logistics)

        self.masking_dict = {}
    def init_masks(self):
        num_hiddens = len(self.hidden_sizes)
        ordering = np.arange(self.flatten_shape)
        self.masking_dict[-1] = ordering

        for h in range(num_hiddens):
            self.masking_dict[h] = np.random.randint(low=self.masking_dict[h-1].min(), high=self.flatten_shape-1, size=self.hidden_sizes[h])
        
        masks = [self.masking_dict[h-1][:, None] <= self.masking_dict[h][None,:] for h in range(num_hiddens)]
        masks.append(self.masking_dict[num_hiddens - 1][:,None] < self.masking_dict[-1][None, :])

        masks[0] = np.repeat(masks[0], self.num_classes, axis = 0)
        masks[-1] = np.repeat(masks[-1], self.num_classes, axis = 1)
        





